In [16]:
# 1. Select a pool of large, liquid, optionable, dividend-yielding stocks
# 2. Collect the expected dividend and ex-dividend dates of these stocks
# 3. Select an options month that is after the ex-dividend date
# 4. Select an OTM option that provides a balance of liquidity, upside, and credit

In [21]:
import pandas as pd

roundup_df = pd.read_csv("2018-09-25_blue-chips.csv")
roundup_df.columns

Index(['Ticker', 'No.', 'Market Cap', 'Dividend %', 'ROA %', 'ROE %', 'ROI %',
       'Curr R', 'Quick R', 'LTDebt/Eq', 'Debt/Eq', 'Gross M %', 'Oper M %',
       'Profit M %', 'Earnings', 'Price', 'Change %', 'Volume',
       'Call Expiration', 'Call Last', 'Call Chg', 'Call Bid', 'Call Ask',
       'Call Vol', 'Call Open', 'Call Strike', 'Ex-Div Date (est)',
       'Div Amount (est)', 'Dividend Period', 'Put Strike', 'Put Expiration',
       'Put Last', 'Put Chg', 'Put Bid', 'Put Ask', 'Put Vol', 'Put Open'],
      dtype='object')

In [42]:
import datetime
import numpy as np
focus_cols = ["Ticker", "Market Cap", "Earnings (est)",
              "Ex-Div Date (est)", "Price", "Div Amount (est)",
              "Call Expiration", "Call Strike", "Call Last",
             "Put Expiration", "Put Strike", "Put Last"]

roundup_df["Earnings"] = pd.to_datetime(roundup_df["Earnings"])
earnings_proj = roundup_df["Earnings"] + roundup_df["Dividend Period"].apply(lambda d: datetime.timedelta(days=d))
today = datetime.date.today()
roundup_df["Earnings (est)"] = np.where(roundup_df["Earnings"] < today, earnings_proj, roundup_df["Earnings"])

focused = roundup_df[focus_cols].copy()
focused = focused[~focused["Put Strike"].isnull() & ~focused["Call Strike"].isnull()]
focused

,Ticker,Market Cap,Earnings (est),Ex-Div Date (est),Price,Div Amount (est),Call Expiration,Call Strike,Call Last,Put Expiration,Put Strike,Put Last
1,RDS-A,2.891700e+11,2018-10-25,2018-11-08,69.60,0.799,2019-01-18,75.0,0.70,2018-10-19,65.0,0.14
3,XOM,3.694400e+11,2018-10-26,2018-11-09,86.50,0.820,2018-12-21,95.0,0.23,2018-10-12,81.5,0.11
4,CVX,2.330600e+11,2018-10-26,2018-11-15,123.37,1.120,2018-12-21,135.0,0.64,2018-11-02,119.0,1.29
6,PG,2.086400e+11,2018-10-19,2018-10-18,83.12,0.717,2018-11-16,87.5,0.36,2018-10-05,80.0,0.04
7,WFC,2.638400e+11,2018-10-12,2018-11-15,54.23,0.430,2019-01-18,57.5,1.15,2018-11-02,49.0,0.15
8,PFE,2.568700e+11,2018-10-30,2018-10-25,43.79,0.340,2018-11-16,47.0,0.15,2018-10-05,40.0,0.03
9,INTC,2.141400e+11,2018-10-25,2018-11-06,45.91,0.300,2018-12-21,50.0,0.76,2018-10-19,42.0,0.13
10,JNJ,3.718700e+11,2018-10-16,2018-11-29,138.64,0.900,2018-12-21,150.0,0.62,2018-11-16,125.0,0.34
11,WMT,2.780900e+11,2018-12-13,2018-12-06,95.10,0.520,2019-01-18,100.0,2.12,2018-11-16,87.5,0.72
12,HD,2.358800e+11,2018-11-13,2018-11-28,207.46,1.030,2018-12-21,220.0,2.52,2018-11-16,190.0,1.05


In [45]:
perf = focused[["Ticker", "Earnings (est)", "Ex-Div Date (est)"]].copy()
perf["P Margin"] = focused["Put Strike"] * 100
perf["P Credit %"] = focused["Put Last"] / focused["Put Strike"] * 100
perf["P Downside %"] = (focused["Price"] - focused["Put Strike"]) / focused["Price"] * 100
perf["CC Margin"] = (focused["Price"] - focused["Call Last"]) * 100
perf["D Credit %"] = focused["Div Amount (est)"] * 100 / perf["CC Margin"] * 100
perf["C Credit %"] = focused["Call Last"] * 100 / perf["CC Margin"] * 100
perf["C Upside %"] = (focused["Call Strike"] - focused["Price"]) / focused["Price"] * 100
# focused["DTE"] = (focused["Call Expiration"] - today).astype('timedelta64[D]')
# focused["Early Exercise Days"] = (focused["Call Expiration"] - focused["Ex-Div Date (est)"]).astype('timedelta64[D]')

perf

,Ticker,Earnings (est),Ex-Div Date (est),P Margin,P Credit %,P Downside %,CC Margin,D Credit %,C Credit %,C Upside %
1,RDS-A,2018-10-25,2018-11-08,6500,0.215385,6.609195,6890,1.159652,1.015965,7.758621
3,XOM,2018-10-26,2018-11-09,8150,0.134969,5.780347,8627,0.950504,0.266605,9.826590
4,CVX,2018-10-26,2018-11-15,11900,1.084034,3.542190,12273,0.912572,0.521470,9.426927
6,PG,2018-10-19,2018-10-18,8000,0.050000,3.753609,8276,0.866361,0.434993,5.269490
7,WFC,2018-10-12,2018-11-15,4900,0.306122,9.644108,5308,0.810098,2.166541,6.029873
8,PFE,2018-10-30,2018-10-25,4000,0.075000,8.654944,4364,0.779102,0.343721,7.330441
9,INTC,2018-10-25,2018-11-06,4200,0.309524,8.516663,4515,0.664452,1.683278,8.908734
10,JNJ,2018-10-16,2018-11-29,12500,0.272000,9.838430,13802,0.652079,0.449210,8.193883
11,WMT,2018-12-13,2018-12-06,8750,0.822857,7.991588,9298,0.559260,2.280060,5.152471
12,HD,2018-11-13,2018-11-28,19000,0.552632,8.416080,20494,0.502586,1.229628,6.044539
